In [1]:
import numpy as np
import pandas as pd
from typing import Callable, List

## Задание № 7

## Вариант 2

## Найти решение Задачи Коши для уравнения $y'(x) = f(x,y)$ с начальными условиями $y(x_0) = y_0$

## $y'(x) = -y(x) + x$, $y(0) = 1$, $h=0.1$, $N = 10$

In [2]:
def func(x: float, y: float) -> float:
    return -y + x

## Общее решение  $y = \frac{2}{e^x} + x - 1$

In [3]:
def solve(x: float) -> float:
    return 2 / np.exp(x) + x - 1

In [4]:
x_0 = 0
y_0 = 1

In [5]:
N = int(input())
h = float(input())

10
0.1


In [6]:
node = np.array([x_0 + h * i for i in range(-2, N + 1)])
node

array([-0.2, -0.1,  0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,
        0.9,  1. ])

### Разложим в ряд Тейлора до 5 ненулевых членов

### $y(0) = 1$

### $y'(0) = -y(0) + 0 = -1$    

### $y''(0) = -y'(0) = 1$

### $y^{(3)}(0) =  -y''(0) = -1$

### $y^{(4)}(0) = 1$

### $y^{(5)}(0) = -1$

### $y_M(x) = 1 - x + x^2 - \frac{1}{3}x^3 + \frac{1}{12}x^4 - \frac{1}{60}x^5$

In [7]:
def taylor(arg: float) -> float:
    return 1 - arg + arg**2 - 1/3 * arg**3 + 1/12 * arg**4 - 1/60 * arg**5

## Найдем решение методом разложения в ряд Тейлора

In [8]:
def method_taylor(taylor: Callable, nodes: np.array):
    values = []
    for node in nodes:
        values.append(taylor(node))
    nodes_values = np.array([nodes, np.array(values)])
    return nodes_values

In [9]:
node_value0 = method_taylor(taylor, node)

In [10]:
correct_value0 = solve(node_value0[0])

In [11]:
df = pd.DataFrame(np.array([node_value0[0],
                           node_value0[1],
                           correct_value0,
                           correct_value0 - node_value0[1]]),
                  index = ['Node', 
                           'Value',
                           'Correct value',
                           'Error'])

In [12]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
Node,-2.000000e-01,-1.000000e-01,0.0,1.000000e-01,2.000000e-01,0.300000,0.400000,0.500000,0.600000,0.700000,0.800000,0.900000,1.000000
Value,1.242805e+00,1.110342e+00,1.0,9.096748e-01,8.374613e-01,0.781635,0.740629,0.713021,0.697504,0.692874,0.698005,0.711834,0.733333
Correct value,1.242806e+00,1.110342e+00,1.0,9.096748e-01,8.374615e-01,0.781636,0.740640,0.713061,0.697623,0.693171,0.698658,0.713139,0.735759
Error,1.829870e-07,2.817962e-09,0.0,2.738586e-09,1.728226e-07,0.000002,0.000011,0.000040,0.000119,0.000297,0.000653,0.001306,0.002426


## Метод Адамса решения задачи Коши для ОДУ

In [13]:
def get_dd_table(lst: List[float]) -> np.ndarray:
    
    result = []
    result.append(lst)
    for i in range(1, 5):
        current_lst = [result[i-1][j+1] - result[i-1][j] for j in range(0, len(result[i-1])-1)]
        result.append(current_lst)
    return result

In [14]:
def get_ext_adams_table(func: Callable, N: int, h: float, x0: float, y0: float, first_node, first_value) -> pd.DataFrame:
    
    nodes = list(first_node)
    values = list(first_value)
    eta_lst = [h * func(nodes[i], values[i]) for i in range(len(nodes))]
    
    dd_table = get_dd_table(eta_lst)

    coeffs = [1., 1 / 2, 5 / 12, 3 / 8, 251 / 720]
    new_value = values[-1] + sum([dd_table[i][-1] * coeffs[i] for i in range(0, 5)])

    
    values.append(new_value)
    nodes.append(nodes[-1] + h)
    
    for i in range(6, N + 3):
        temp = h * func(nodes[-1], values[-1])
        dd_table[0].append(temp)
        for j in range(1, 5):
            dd_table[j].append(dd_table[j-1][-1] - dd_table[j-1][-2])
        
        new_value = values[-1] + sum([dd_table[i][-1] * coeffs[i] for i in range(0, 5)])

        
        values.append(new_value)
        nodes.append(nodes[-1] + h)

    return [nodes, values]
      

In [15]:
node_value1 = get_ext_adams_table(func, N, h, x_0, y_0, node_value0[0][:5], node_value0[1][:5])

In [16]:
df = pd.DataFrame(np.array([node_value1[0],
                           node_value1[1],
                           correct_value0,
                           correct_value0 - node_value1[1]]),
                  index = ['Node', 
                           'Value',
                           'Correct value',
                           'Error'])

In [17]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
Node,-2.000000e-01,-1.000000e-01,0.0,1.000000e-01,2.000000e-01,3.000000e-01,0.400000,0.500000,0.600000,0.700000,0.800000,0.900000,1.000000
Value,1.242805e+00,1.110342e+00,1.0,9.096748e-01,8.374613e-01,7.816357e-01,0.740639,0.713060,0.697621,0.693168,0.698656,0.713137,0.735756
Correct value,1.242806e+00,1.110342e+00,1.0,9.096748e-01,8.374615e-01,7.816364e-01,0.740640,0.713061,0.697623,0.693171,0.698658,0.713139,0.735759
Error,1.829870e-07,2.817962e-09,0.0,2.738586e-09,1.728226e-07,7.538634e-07,0.000001,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002


## Метод Эйлера

### $y_{k+1} = y_k + hf(x_k, y_k)$

In [18]:
def method_euler(
    func: Callable,
    x_0: float,
    y_0: float,
    h: float,
    N: int
    ):
    
    node_value = [[x_0], [y_0]]
    x = x_0
    y = y_0
    for _ in range(N):
        y = y + h * func(x, y)
        x += h
        node_value[0].append(x)
        node_value[1].append(y)
    node_value = np.array([np.array(node_value[0]), np.array(node_value[1])])
    return node_value

In [19]:
node_value = method_euler(func,x_0, y_0, h, N)

In [20]:
correct_value = solve(node_value[0])

In [21]:
df = pd.DataFrame(np.array([node_value[0],
                           node_value[1],
                           correct_value,
                           correct_value - node_value[1]]),
                  index = ['Node', 
                           'Value',
                           'Correct value',
                           'Error'])

In [22]:
df

,0,1,2,3,4,5,6,7,8,9,10
Node,0.0,0.100000,0.200000,0.300000,0.40000,0.500000,0.600000,0.700000,0.800000,0.900000,1.000000
Value,1.0,0.900000,0.820000,0.758000,0.71220,0.680980,0.662882,0.656594,0.660934,0.674841,0.697357
Correct value,1.0,0.909675,0.837462,0.781636,0.74064,0.713061,0.697623,0.693171,0.698658,0.713139,0.735759
Error,0.0,0.009675,0.017462,0.023636,0.02844,0.032081,0.034741,0.036577,0.037724,0.038298,0.038402


## Метод Эйлера. Модификация 1

   ### I) $y_{k+1/2} = y_k + \frac{h}{2}f(x_k, y_k)$

### II) $y_{k+1} = y_k + hf(x_k + \frac{h}{2}, y_{k + 1/2})$

In [23]:
def method_euler_mod1(
    func: Callable,
    x_0: float,
    y_0: float,
    h: float,
    N: int
    ):
    
    node_value = [[x_0], [y_0]]
    x = x_0
    y = y_0
    for _ in range(N):
        y_  = y + h/2 * func(x, y)
        y = y + h * func(x + h/2, y_)
        x += h
        node_value[0].append(x)
        node_value[1].append(y)
    node_value = np.array([np.array(node_value[0]), np.array(node_value[1])])
    return node_value

In [24]:
node_value1 = method_euler_mod1(func,x_0, y_0, h, N)

In [25]:
correct_value1 = solve(node_value1[0])

In [26]:
df = pd.DataFrame(np.array([node_value1[0],
                           node_value1[1],
                           correct_value1,
                           correct_value1 - node_value1[1]]),
                  index = ['Node', 
                           'Value',
                           'Correct value',
                           'Error'])

In [27]:
df

,0,1,2,3,4,5,6,7,8,9,10
Node,0.0,0.100000,0.200000,0.300000,0.400000,0.500000,0.600000,0.700000,0.800000,0.900000,1.000000
Value,1.0,0.910000,0.838050,0.782435,0.741604,0.714152,0.698807,0.694420,0.699951,0.714455,0.737082
Correct value,1.0,0.909675,0.837462,0.781636,0.740640,0.713061,0.697623,0.693171,0.698658,0.713139,0.735759
Error,0.0,-0.000325,-0.000588,-0.000799,-0.000964,-0.001090,-0.001184,-0.001250,-0.001293,-0.001316,-0.001323


## Метод Эйлера. Модификация 2

### $Y_{k+1} = y_k + h f(x_k, y_k)$

### $y_{k+1} = y_k + \frac{h}{2} [f(x_k, y_k) + f(x_{k+1}, Y_{k+1})]$

In [28]:
def method_second(
    func: Callable,
    x_0: float,
    y_0: float,
    h: float,
    N: int
    ):
    
    node_value = [[x_0], [y_0]]
    x = x_0
    y = y_0
    for _ in range(N):
        # 1 
        Y = y + h * func(x, y)
        # 2
        x = x + h
        y = y + h/2 * (func(x - h, y) + func(x, Y))
        node_value[0].append(x)
        node_value[1].append(y)
    node_value = np.array([np.array(node_value[0]), np.array(node_value[1])])
    return node_value

In [29]:
node_value2 = method_second(func,x_0, y_0, h, N)

In [30]:
df = pd.DataFrame(np.array([node_value2[0],
                           node_value2[1],
                           correct_value1,
                           correct_value1 - node_value2[1]]),
                  index = ['Node', 
                           'Value',
                           'Correct value',
                           'Error'])

In [31]:
df

,0,1,2,3,4,5,6,7,8,9,10
Node,0.0,0.100000,0.200000,0.300000,0.400000,0.500000,0.600000,0.700000,0.800000,0.900000,1.000000
Value,1.0,0.910000,0.838050,0.782435,0.741604,0.714152,0.698807,0.694420,0.699951,0.714455,0.737082
Correct value,1.0,0.909675,0.837462,0.781636,0.740640,0.713061,0.697623,0.693171,0.698658,0.713139,0.735759
Error,0.0,-0.000325,-0.000588,-0.000799,-0.000964,-0.001090,-0.001184,-0.001250,-0.001293,-0.001316,-0.001323


## Метод Рунге-Кутта

In [32]:
def method_runge_kutta(
    func: Callable,
    x_0: float,
    y_0: float,
    h: float,
    N: int
    ):
    
    node_value = [[x_0], [y_0]]
    x = x_0
    y = y_0
    for _ in range(N):
        # 4 constant
        
        k_1 = h * func(x,y)
        k_2 = h * func(x + h/2,y + k_1/2)
        k_3 = h * func(x + h/2,y + k_2/2)
        k_4 = h * func(x + h,y + k_3)
        
        # increment
        
        x += h
        y += 1/6*(k_1 + 2 * k_2 + 2 *k_3 + k_4)
        node_value[0].append(x)
        node_value[1].append(y)
    node_value = np.array([np.array(node_value[0]), np.array(node_value[1])])
    return node_value

In [33]:
node_value3 = method_runge_kutta(func,x_0, y_0, h, N)

In [34]:
df = pd.DataFrame(np.array([node_value3[0],
                           node_value3[1],
                           correct_value1,
                           correct_value1 - node_value3[1]]),
                  index = ['Node', 
                           'Value',
                           'Correct value',
                           'Error'])

In [35]:
df

,0,1,2,3,4,5,6,7,8,9,10
Node,0.0,1.000000e-01,2.000000e-01,3.000000e-01,4.000000e-01,5.000000e-01,6.000000e-01,7.000000e-01,8.000000e-01,9.000000e-01,1.000000e+00
Value,1.0,9.096750e-01,8.374618e-01,7.816368e-01,7.406406e-01,7.130619e-01,6.976239e-01,6.931712e-01,6.986586e-01,7.131400e-01,7.357595e-01
Correct value,1.0,9.096748e-01,8.374615e-01,7.816364e-01,7.406401e-01,7.130613e-01,6.976233e-01,6.931706e-01,6.986579e-01,7.131393e-01,7.357589e-01
Error,0.0,-1.639281e-07,-2.966565e-07,-4.026389e-07,-4.857637e-07,-5.494215e-07,-5.965646e-07,-6.297596e-07,-6.512344e-07,-6.629190e-07,-6.664821e-07
